In [ ]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

In [ ]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [ ]:
url ="https://www.anantara.com/en/al-baleed-salalah"
url ="https://www.ghmhotels.com/en/muscat/"
url = "https://www.hotelpalazzogiovanelli.com/"

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept-Language": "en-GB,en;q=0.5",
    "Referer": "https://google.com",
    "DNT": "1"
}

In [ ]:
loader = RecursiveUrlLoader(
    url=url, max_depth=2, headers=headers, extractor=lambda x: Soup(x, "html.parser").text
)
documents = loader.load()

In [ ]:
import re
# Remove 2 or more consecutive \n with a single \n in each document
for doc in documents:
    doc.page_content = re.sub(r'\n{2,}', '\n', doc.page_content)

# Remove 2 or more consecutive \n with a single \n in each document
for doc in documents:
    doc.page_content = re.sub(r'\t{2,}', '\t', doc.page_content)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
db = FAISS.from_documents(texts, embeddings)

In [ ]:
# retriever = db.as_retriever()

In [ ]:
retriever = db.as_retriever(search_type="mmr")

In [ ]:
# retriever = db.as_retriever(search_kwargs={"k": 1})

In [ ]:
# 1. hotel name

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

docs = retriever.get_relevant_documents("Hotel name")

# Define prompt
prompt_template = """What is the name of the hotel:
"{text}"
Return only the hotel name:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs))

In [ ]:


from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI

class Hotel(BaseModel):
    hotel_name: str


# llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
runnable = create_structured_output_runnable(Hotel, llm)

hotel_name = runnable.invoke(documents[0].page_content)

In [ ]:
hotel_name.hotel_name

In [ ]:
# Define prompt
prompt_template = """What is the name of the hotel:
"{text}"
Return only the hotel name:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(documents[:1]))

In [ ]:
# 2. Short introduction

In [ ]:
short = "Short introduction - The purpose of this short introduction is to explain succinctly why we’d recommend this hotel: why have we hand-picked it? It should highlight the main benefits of staying at the hotel."
docs = retriever.get_relevant_documents(short)
# Define prompt
prompt_template = """ Write a short introduction of the hotel using the following guidelines and relevant text from the hotel website.
Word couunt for short introduction: ~ 30 - 40 words
The purpose of this short introduction is to explain succinctly why we’d recommend this hotel: why have we hand-picked it? It should highlight the main benefits of staying at the hotel. 
Tip:  Avoid time-sensitive references that could soon become out- of-date, for example ‘brand -new hotel’. 
"{text}"
Helpful answer:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs))

In [ ]:
texts[0].page_content

In [ ]:
# 3. Main hotel introduction

In [ ]:
long = "Main Hotel introduction - This wording for is the starting paragraphs of the main body text about the hotel. It expands on the reasons why we’ve chosen this hotel, which the short intro touches upon. It elaborates on the style and feel of the hotel, rather than specific features, which are liable to change. It can include any information that is essential for documentation."
docs = retriever.get_relevant_documents(long)

# Define prompt
prompt_template = """ You are a content writer for a tour operator writing hotel descriptions for our own website, not the hotels.
Write the Main Hotel introduction of the hotel using the following guidelines and relevant text from the hotel website.
This wording for is the starting paragraphs of the main body text about the hotel. It expands on the reasons why we’ve chosen this hotel, which the short intro touches upon. It elaborates on the style and feel of the hotel, rather than specific features, which are liable to change. It can include any information that is essential for documentation.     
Tip: Don’t get too specific as details can change, for example avoid referencing the number of rooms
"{text}"
Helpful answer:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs))

In [ ]:
# 4. Location

In [ ]:
loctation = "Location - This section focuses on the benefits of the location but also makes the reader aware of any pragmatic considerations. It can mention nearby attractions or the nearest areas for eating and hopping, and how long it takes to reach them."
docs = retriever.get_relevant_documents(loctation)

# Define prompt
prompt_template = """ Write a summary of a hotels location using the following guidelines and relevant text from the hotel website.
Word count for location: ~ 60 words
This section focuses on the benefits of the location but also makes the reader aware of any pragmatic considerations. It can mention nearby attractions or the nearest areas for eating and hopping, and how long i t takes t o reach them.:
"{text}"
Helpful answer:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(texts))

In [ ]:
# 5. Room intro

In [ ]:

room_intro = "Room intro - An easy-reference section that displays  the top-level details on the room choices of the hotel. It shouldn’t  drill into too much detail, to save  the information becoming out of date, but aim to provide the most important features. Confirm the following room  facilities/amenities: en suite bathroom (but not bath v shower); air conditioning; Wi-Fi."
docs = retriever.get_relevant_documents(room_intro)

# Define prompt
prompt_template = """ Write a summary of a hotels room choices using the following guidelines and relevant text from the hotel website.
Word count for room  intro:  ~ 100 words
An easy-reference section that displays  the top-level details on the room choices of the hotel. It shouldn’t  drill into too much detail, to save  the information becoming out of date, but aim to provide the most important features. Confirm the following room  facilities/amenities: en suite bathroom (but not bath v shower); air conditioning; Wi-Fi. 
"{text}"
Helpful answer:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs))

In [ ]:
# 6. Food & drink

In [ ]:
food_drink = "Food & drink - An easy-reference bullet section that lists the top-level eating and drinking options in the hotel. It shouldn’t drill into too much detail, to save the information becoming out of date, but aim to provide the most important features. A starting set of bullet points, which you can adapt and add to, is in the hotel writing template, which you should use alongside this guidance."
docs = retriever.get_relevant_documents(food_drink)

# Define prompt
prompt_template = """ Write a summary of a hotels Food & drink options using the following guidelines and relevant text from the hotel website.
Word count for eating & drinking: ~ 60 words
An easy-reference bullet section that lists the top-level eating and drinking options in the hotel. It shouldn’t drill into too much detail, to save the information becoming out of date, but aim to provide the most important features. A starting set of bullet points, which you can adapt and add to, is in the hotel writing template, which you should use alongside this guidance.
"{text}"
Helpful answer:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(texts))

In [ ]:
# 7. Facilities and Activities

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

Facilities_activities = "Facilities and Activities - An easy-reference bullet section that lists the key facilities the hotel offers, such as swimming pool/s, laundry, spa, and so on. Also include any unusual or bespoke features, such as the hot springs at Colca Lodge in Peru . Starting suggestions for the bullets are in the hotel writing template , which you should use alongside this guidance."
docs = retriever.get_relevant_documents(Facilities_activities)

# Define prompt
prompt_template = """ Write a summary of a hotels Facilities and Activities options using the following guidelines and relevant text from the hotel website.
Word count for facilities: ~ 60 words
An easy-reference bullet section that lists the key facilities the hotel offers, such as swimming pool/s, laundry, spa, and so on. Also include any unusual or bespoke features, such as the hot springs at Colca Lodge in Peru . Starting suggestions for the bullets are in the hotel writing template , which you should use alongside this guidance.
"{text}"
Helpful answer:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(texts))